In [ ]:
%pip install selenium
%pip install beautifulsoup4
%pip install plotly
%pip install seaborn
%pip install scipy
%pip install nbformat

In [71]:
%pip freeze | findstr nbformat


nbformat==5.10.4
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
from selenium.webdriver.chrome.service import Service

download_service = Service()

driver = webdriver.Chrome(service=download_service)

#driver.maximize_window()

The chromedriver version (131.0.6778.69) detected in PATH at I:\ChromeDriver\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.83); currently, chromedriver 132.0.6834.83 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


In [3]:
# Otwieranie strony Steam
url = "https://store.steampowered.com/category/science_fiction/"
driver.get(url)

# Czekanie na załadowanie treści
time.sleep(2)  # Dostosuj, jeśli strona ładuje się szybciej

In [48]:
driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')


In [29]:
load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
load_more_button.click()

In [4]:
def GetTitlesWithTagsAndPrices():
    games = []
    game_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]")

    for game in game_containers:
        try:
            # Pobranie tytułu gry
            title = game.find_element(By.XPATH, ".//div[contains(@class, '_3rrH9dPdtHVRMzAEw82AId')]//a//div").text.strip()

            # Pobranie tagów
            tags_container = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]")
            tag_elements = tags_container.find_elements(By.XPATH, ".//div[contains(@class, '_2bkP-3b7dvr0a_qPdZEfHY')]//a")
            tags = [tag.text.strip() for tag in tag_elements if tag.text.strip()]

            # Pobranie ceny gry
            try:
                price_container = game.find_element(By.XPATH, ".//div[contains(@class, '_1JuIpzMtS7-xZrnUmEQ4my')]//div[contains(@class, '_2s-O5T3qJJYR2AUq4b9jIN')]")
                price = price_container.find_element(By.XPATH, ".//div[contains(@class, '_3j4dI1yA7cRfCvK8h406OB')]").text.strip()
            except Exception as e:
                price = "Brak ceny"  # Obsługa gier bez podanej ceny

            # Dodanie gry do listy
            games.append({
                "title": title,
                "tags": tags,
                "price": price
            })
        except Exception as e:
            print(f"Problem z przetwarzaniem gry: {e}")

    # Wyświetlenie wyników
    print("Lista gier, ich tagów i cen:")
    for game in games:
        print(f"Tytuł: {game['title']}, Tagi: {', '.join(game['tags'])}, Cena: {game['price']}")

    return games


In [5]:
# zescrolluj do początku listy
#driver.execute_script('window.scroll(0, document.body.scrollHeight-3000)')

# zecrolluj na dół listy
driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')

# Powtarzanie procesu 10 razy
for i in range(25):
    load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
    load_more_button.click()
    time.sleep(2)
    driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')


data = GetTitlesWithTagsAndPrices()

Lista gier, ich tagów i cen:
Tytuł: Marvel Rivals, Tagi: Free to Play, Wieloosobowe, Strzelanka z bohaterami, Cena: Free to Play
Tytuł: Warhammer 40,000: Space Marine 2, Tagi: Warhammer 40K, Akcja, Strzelanka z perspektywą trzeciej osoby, Wieloosobowe, Cena: 157,49 zł
Tytuł: Detroit: Become Human, Tagi: Znaczące wybory, Bogata fabuła, Wiele zakończeń, Filmowe, Jednoosobowe, Cena: 55,49 zł
Tytuł: Stellaris, Tagi: Kosmos, Strategiczne globalne, Strategiczne, Science fiction, 4X, Cena: 46,24 zł
Tytuł: HELLDIVERS™ 2, Tagi: Sieciowa kooperacja, PvE, Strzelanka z perspektywą trzeciej osoby, Wieloosobowe, Akcja, Cena: 169,00 zł
Tytuł: Cyberpunk 2077, Tagi: Cyberpunk, Otwarty świat, Nagość, RPG, Jednoosobowe, Cena: 199,00 zł
Tytuł: Among Us, Tagi: Wieloosobowe, Sieciowa kooperacja, Dedukcyjne towarzyskie, Kosmos, Survival, Cena: 10,79 zł
Tytuł: Apex Legends™, Tagi: Free to Play, Battle royale, Wieloosobowe, FPS, Strzelanka, Cena: Free to Play
Tytuł: Warframe, Tagi: Free to Play, RPG akcji, Str

In [6]:
# Zamknięcie drivera
driver.quit()

In [7]:
import numpy as np

def clean_data(df):
    # Konwersja ceny do liczby
    def clean_price(price):
        try:
            if "Free to Play" in price:
                return 0.0
            return float(price.replace(' zł', '').replace(',', '.'))
        except:
            return np.nan

    # Rozbijanie tagów
    def split_tags(tags):
        return tags.split(', ') if isinstance(tags, str) else []

    # Czyszczenie cen
    df['price'] = df['price'].apply(clean_price)
    
    # Usuwanie pustych rekordów
    df.dropna(subset=['price', 'tags'], inplace=True)
    
    # Usuwanie duplikatów na podstawie tytułów
    df.drop_duplicates(subset=['title'], inplace=True)
    
    # Rozbijanie tagów na kolumny (binary encoding)
    all_tags = set(tag for tags in df['tags'].apply(split_tags) for tag in tags)
    for tag in all_tags:
        df[f"Tag_{tag}"] = df['tags'].apply(lambda x: 1 if tag in x else 0)

    return df


In [12]:
display(df.to_string())

"                                                              title                                                                                         tags   price\n0                                                     Marvel Rivals                                        [Free to Play, Wieloosobowe, Strzelanka z bohaterami]    0.00\n1                                  Warhammer 40,000: Space Marine 2                [Warhammer 40K, Akcja, Strzelanka z perspektywą trzeciej osoby, Wieloosobowe]  157.49\n2                                             Detroit: Become Human                     [Znaczące wybory, Bogata fabuła, Wiele zakończeń, Filmowe, Jednoosobowe]   55.49\n3                                                         Stellaris                           [Kosmos, Strategiczne globalne, Strategiczne, Science fiction, 4X]   46.24\n4                                                     HELLDIVERS™ 2     [Sieciowa kooperacja, PvE, Strzelanka z perspektywą trzeciej osoby, Wieloosob

In [8]:
import pandas as pd

# Dane wejściowe
df = pd.DataFrame(data)

# Przetwarzanie danych
df = clean_data(df)


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import plotly.express as px

def analyze_data(df):
    # 1. Tworzenie tabeli krzyżowej tagów
    def create_crosstab(df):
        all_tags = [tag for tags in df['tags'] for tag in tags]  # Zbiór wszystkich tagów
        unique_tags = sorted(set(all_tags))  # Unikalne tagi
        
        # Tworzenie macierzy obecności tagów
        tag_matrix = pd.DataFrame(0, index=unique_tags, columns=unique_tags)
        for tags in df['tags']:
            for tag1 in tags:
                for tag2 in tags:
                    if tag1 != tag2:  # Ignorujemy współwystępowanie z samym sobą
                        tag_matrix.loc[tag1, tag2] += 1
        
        return tag_matrix

    # 2. Test chi-kwadrat na współwystępowanie tagów
    def chi_square_analysis(tag_matrix):
        chi2_results = []
        for tag1 in tag_matrix.index:
            for tag2 in tag_matrix.columns:
                if tag1 != tag2:  # Test tylko dla różnych tagów
                    obs = [[tag_matrix.loc[tag1, tag2], tag_matrix.loc[tag2, tag1]],
                           [sum(tag_matrix.loc[tag1, :]), sum(tag_matrix.loc[:, tag2])]]
                    _, p, _, _ = chi2_contingency(obs)
                    chi2_results.append((tag1, tag2, p))
        
        return pd.DataFrame(chi2_results, columns=['Tag1', 'Tag2', 'P-value']).sort_values('P-value')

    # 3. Analiza wpływu tagów na ceny
    def tag_price_analysis(df):
        tag_price_data = []
        for tag in sorted(set(tag for tags in df['tags'] for tag in tags)):
            filtered = df[df['tags'].apply(lambda tags: tag in tags)]
            avg_price = filtered['price'].mean()
            tag_price_data.append({'Tag': tag, 'Average Price': avg_price})
        
        return pd.DataFrame(tag_price_data).sort_values('Average Price', ascending=False)

    # Tworzenie tabeli krzyżowej tagów
    tag_matrix = create_crosstab(df)
    print("Tabela krzyżowa tagów:\n", tag_matrix)

    # Analiza współwystępowania tagów z testem chi-kwadrat
    #chi2_results = chi_square_analysis(tag_matrix)
    #print("Wyniki testu chi-kwadrat:\n", chi2_results.head(10))

    # Analiza wpływu tagów na ceny
    tag_price_df = tag_price_analysis(df)
    print("Średnia cena dla poszczególnych tagów:\n", tag_price_df)

    # Wizualizacja danych
    fig1 = px.imshow(tag_matrix, text_auto=True, title="Współwystępowanie tagów (Tabela krzyżowa)")
    fig2 = px.bar(tag_price_df, x='Tag', y='Average Price', title="Średnia cena w zależności od tagu", labels={'Tag': 'Tag', 'Average Price': 'Średnia cena'})
    
    fig1.show()
    fig2.show()

    return {
        "tag_matrix": tag_matrix,
        #"chi2_results": chi2_results,
        "tag_price_analysis": tag_price_df
    }


In [11]:
# Analiza
analyze_data(df)


Tabela krzyżowa tagów:
                               2,5D  2D  3D  4X  Akcja  Anime  Automatyzacja  \
2,5D                             0   0   0   0      0      0              0   
2D                               0   0   0   0      2      0              0   
3D                               0   0   0   0      0      1              0   
4X                               0   0   0   0      0      0              0   
Akcja                            0   2   0   0      0      2              0   
...                            ...  ..  ..  ..    ...    ...            ...   
Łamigłówki                       0   1   0   0      0      0              0   
Łamigłówki platformowe           1   0   0   0      0      0              0   
Łupy                             0   0   0   0      0      0              0   
Świetna ścieżka dźwiękowa        0   0   0   0      2      0              0   
Żeńska postać w roli głównej     0   1   1   0      1      0              0   

                           

{'tag_matrix':                               2,5D  2D  3D  4X  Akcja  Anime  Automatyzacja  \
 2,5D                             0   0   0   0      0      0              0   
 2D                               0   0   0   0      2      0              0   
 3D                               0   0   0   0      0      1              0   
 4X                               0   0   0   0      0      0              0   
 Akcja                            0   2   0   0      0      2              0   
 ...                            ...  ..  ..  ..    ...    ...            ...   
 Łamigłówki                       0   1   0   0      0      0              0   
 Łamigłówki platformowe           1   0   0   0      0      0              0   
 Łupy                             0   0   0   0      0      0              0   
 Świetna ścieżka dźwiękowa        0   0   0   0      2      0              0   
 Żeńska postać w roli głównej     0   1   1   0      1      0              0   
 
                        